In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torchvision.transforms as transforms
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
## Torchvision
import torchvision
from torchvision.datasets import STL10
from torchvision import transforms

In [12]:
from torchvision.transforms import ToTensor
from torchvision.datasets import MNIST

transform = ToTensor()
dataset = MNIST(root='./data', download=True, train=True, transform=transform)


In [13]:
print("dataset size:", len(dataset))


dataset size: 60000


In [14]:
dataset_size = len(dataset)
train_size = int(0.6 * dataset_size)
validation_size = int(0.2 * dataset_size)
test_size = dataset_size - train_size - validation_size

print("Train size:", train_size)
print("Validation size:", validation_size)
print("Test size:", test_size)


Train size: 36000
Validation size: 12000
Test size: 12000


In [15]:
from torch.utils.data import random_split

train_dataset, valid_dataset, test_dataset = random_split(dataset, [train_size, validation_size, test_size])


In [16]:
# Set batch size
batch_size = 64

# Create DataLoader instances
trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=True)
testloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [17]:
import torch.nn as nn
import torch.nn.functional as F

class CustomMobileNet(nn.Module):
    def __init__(self): 
        super(CustomMobileNet, self).__init__()
        # Define layers 
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)  
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

custom_model = CustomMobileNet()


In [18]:
import torch.optim as optim
import torch.nn as nn

# Define optimizer and loss criterion
custom_optimizer = optim.Adam(model.parameters(), lr=0.01)
custom_criterion = nn.CrossEntropyLoss()


In [19]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

num_epochs = 10  

for epoch in range(num_epochs):
    
    model.train()
    total_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    for i, data in enumerate(trainloader, 0):
        
        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total_samples += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()

    average_loss = total_loss / len(trainloader)
    accuracy = 100 * correct_predictions / total_samples
    print(f"Epoch {epoch + 1}, Train Loss: {average_loss:.4f}, Train Accuracy: {accuracy:.2f}%")


Epoch 1, Train Loss: 0.0679, Train Accuracy: 98.23%
Epoch 2, Train Loss: 0.0526, Train Accuracy: 98.60%
Epoch 3, Train Loss: 0.0498, Train Accuracy: 98.65%
Epoch 4, Train Loss: 0.0453, Train Accuracy: 98.77%
Epoch 5, Train Loss: 0.0487, Train Accuracy: 98.74%
Epoch 6, Train Loss: 0.0536, Train Accuracy: 98.61%
Epoch 7, Train Loss: 0.0436, Train Accuracy: 98.92%
Epoch 8, Train Loss: 0.0520, Train Accuracy: 98.73%
Epoch 9, Train Loss: 0.0442, Train Accuracy: 98.82%
Epoch 10, Train Loss: 0.0401, Train Accuracy: 98.98%


In [23]:
# Validation 
model.eval()  
total_validation_loss = 0.0
correct_predictions_val = 0
total_samples_val = 0

with torch.no_grad():  
    for data in validloader:  
        images, labels = data[0].to(device), data[1].to(device) 
        outputs = model(images)  
        loss = criterion(outputs, labels)  

        total_validation_loss += loss.item()  
        _, predicted = torch.max(outputs.data, 1) 
        total_samples_val += labels.size(0)  
        correct_predictions_val += (predicted == labels).sum().item() 

validation_loss = total_validation_loss / len(validloader)
validation_accuracy = 100 * correct_predictions_val / total_samples_val

print(f"Epoch {epoch + 1}, Validation Loss: {validation_loss:.4f}, Validation Accuracy: {validation_accuracy:.2f}%")

print('Finished Training')


Epoch 10, Validation Loss: 0.1032, Validation Accuracy: 98.04%
Finished Training


In [22]:
# Testing
model.eval()  
total_test_loss = 0.0
correct_predictions_test = 0
total_samples_test = 0

with torch.no_grad():  
    for data in testloader:  
        images, labels = data[0].to(device), data[1].to(device) 
        outputs = model(images)  
        loss = criterion(outputs, labels) 

        total_test_loss += loss.item()  
        _, predicted = torch.max(outputs.data, 1)  
        total_samples_test += labels.size(0) 
        correct_predictions_test += (predicted == labels).sum().item()  

test_loss = total_test_loss / len(testloader)
test_accuracy = 100 * correct_predictions_test / total_samples_test

print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%")


Test Loss: 0.0837, Test Accuracy: 98.24%
